# Build a taskbook

A taskbook is a collection of tasks and their records.

Whenever a task is executed within a taskbook, its'
arguments, result and other relevant information are recorded.

The main components of an taskbook are the following:

* **Task**:

    `@task`: A decorator to mark a function as a task

* **Taskbook**:

    `@taskbook`: A decorator to convert a function into a taskbook

    `TaskBook`: Result of an executed function decorated with `@taskbook`


## Building a simple taskbook

### Import relevant objects

In [ ]:
from laboneq_applications.workflow import task, taskbook

### Define the tasks

In [ ]:
@task
def measure() -> int:
    return 100


@task
def analyze(measurement_result: int, threshold: int) -> bool:
    return measurement_result < threshold

### Define the taskbook

In [ ]:
@taskbook
def experiment(threshold: int):
    measurement = measure()
    analysis_result = analyze(measurement, threshold)
    if analysis_result:
        return "PASS"
    return "FAIL"

### Run the taskbook

In [ ]:
book = experiment(threshold=101)

### Inspecting the results

#### Inspecting the `TaskBook`

In [ ]:
book

In [ ]:
book.output

#### Inspecting the tasks

In [ ]:
book.tasks

There are several ways to get the individual tasks from the `TaskBook`

Single task with an index or name

In [ ]:
book.tasks[1], book.tasks["analyze"]

Specific task lookup with indexing

The first argument is the name of the task and the second is an integer or a
`slice`

In [ ]:
book.tasks["analyze", :]  # All tasks named 'analyze'

### Rerunning a task

A task can be rerun from the `TaskBook` 

This can be useful when one wants to modify only specific arguments of the task,
e.g a plotting arguments.

In this example we will only lower the `threshold` of the `analyze` task 
and since we did not supply the argument `measurement_result`, the original value
is used and our `analyze` should now fail.

In [ ]:
book.tasks["analyze"].rerun(threshold=99)

After rerunning the task, we can also see that the `TaskBook` is updated with
the rerun results

In [ ]:
book.tasks["analyze", :]

### Run until a task

A `TaskBook` can be run only up to (and including) one of its tasks.

This can be useful if one wants to run only the first few tasks and inspect the result. For example, for an experiment `TaskBook`, one often wants to check if the experiment sequence compiles successfully before running the experiment.

In this example, we will exclude the `analyze` task to first inspect if the `measure` task was successful. To do this, we need to allow `experiment` to accept an options dictionary and then pass the taskbook option `run_until`. We can also use options classes as shown in the [`options.ipynb`](options.ipynb) guide. 

In [ ]:
@taskbook
def experiment(threshold: int, options: dict):
    measurement = measure()
    analysis_result = analyze(measurement, threshold)
    if analysis_result:
        return "PASS"
    return "FAIL"

In [ ]:
options = {"taskbook.run_until": "measure"}
book = experiment(threshold=101, options=options)

In [ ]:
book

### Inspect a `TaskBook` that has failed

In case there is an error during the execution of a `TaskBook`, we can still inspect the tasks that have run up to the task that triggered the error using `recover()`. Note that `recover()` stores only one execution result and can only be called once; a second call to `recover()` raises an exception.

For experiment `TaskBooks`, this is useful for debugging a failed compilation task by inspecting the experiment sequence produced by the previous task. 

In this example, we will add an assertion error to the `analyze` task.

In [ ]:
@task
def measure() -> int:
    return 100


@task
def analyze(measurement_result: int, threshold: int) -> bool:
    # let's add an error in this task
    if not (measurement_result >= 100 and threshold >= 100):
        raise RuntimeError("Something went wrong.")
    return measurement_result < threshold


@taskbook
def experiment(threshold: int):
    measurement = measure()
    analysis_result = analyze(measurement, threshold)
    if analysis_result:
        return "PASS"
    return "FAIL"

In [ ]:
book = experiment(99)

In [ ]:
recovered_book = experiment.recover()
recovered_book

In [ ]:
# Check that the measure task returns a result that is >= 100
recovered_book.tasks["measure"].output

In [ ]:
# Check the value of the threshold passed to the taskbook
recovered_book.input

In [ ]:
# Now we know we have to increase the value of the threshold
book = experiment(101)
book.output

### Defining taskbook options

In [ ]:
from laboneq_applications.workflow import TaskBookOptions

